# Image Processing Pipeline using Snowflake Cortex

This notebooks demonstrates the implementation of an image processing pipeline using [Streams](https://docs.snowflake.com/en/user-guide/streams-intro), [Tasks](https://docs.snowflake.com/en/user-guide/tasks-intro) and [SNOWFLAKE.CORTEX.COMPLETE multimodal](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex-multimodal) capability. (*Currently in Public Preview.*)

Step 1: Create Snowflake managed stage to store sample images.

In [ ]:
CREATE stage GENAI_IMAGES encryption = (TYPE = 'SNOWFLAKE_SSE') directory = ( ENABLE = true );

Step 2: Download two sample images provided below and upload them on stage `GENAI_IMAGES`. [Learn how](https://docs.snowflake.com/en/user-guide/data-load-local-file-system-stage-ui?_fsi=oZm563yp&_fsi=oZm563yp#upload-files-onto-a-named-internal-stage)

Sample images:
- https://sfquickstarts.s3.us-west-1.amazonaws.com/misc/images/other/sample-img-1.png
- https://sfquickstarts.s3.us-west-1.amazonaws.com/misc/images/other/sample-img-2.jpg


*Note: Sample images provided courtesy of [Dash](https://natureunraveled.com/).*

Step 3: Create Stream `images_stream` on stage `GENAI_IMAGES` to detect changes.

In [ ]:
CREATE OR REPLACE STREAM images_stream ON STAGE GENAI_IMAGES;

Step 4: Create target table `image_analysis` to store image analysis.

In [ ]:
CREATE OR REPLACE TABLE image_analysis 
as 
SELECT RELATIVE_PATH,SNOWFLAKE.CORTEX.COMPLETE('pixtral-large',
    'Put image filename in an attribute called "Image."
     Put a short title in title case in an attribute called "Title".
     Put 3 key points in an attribute called "Highlights",
     Extract any statistics (years, historical data, amounts, etc.) along with the context in an attribute called "Stats",
     Put a 200-word detailed summary summarizing the image in an attribute called "Summary"', 
     TO_FILE('@GENAI_IMAGES', RELATIVE_PATH)) as image_classification 
from directory(@GENAI_IMAGES);

Step 5: Preview image analysis produced on the sample images

In [ ]:
select * from image_analysis;

Step 6: Create Task `image_analysis_task` to process new images uploaded on stage `GENAI_IMAGES` using SNOWFLAKE.CORTEX.COMPLETE() multimodal capability.

In [ ]:
CREATE OR REPLACE TASK image_analysis_task
SCHEDULE = '1 minute'
WHEN
  SYSTEM$STREAM_HAS_DATA('images_stream')
AS
  INSERT INTO image_analysis (RELATIVE_PATH, image_classification)
    SELECT RELATIVE_PATH,SNOWFLAKE.CORTEX.COMPLETE('pixtral-large',
        'Put image filename in an attribute called "Image."
         Put a short title in title case in an attribute called "Title".
         Put a 200-word detailed summary summarizing the image in an attribute called "Summary"', 
         TO_FILE('@GENAI_IMAGES', RELATIVE_PATH)) as image_classification 
    from images_stream;

-- NOTE: Tasks are suspended by default so let's resume it.
ALTER TASK image_analysis_task RESUME;

Step 7: Confirm Task status 

In [ ]:
SHOW TASKS like 'image_analysis_task';

Step 8:  Download new sample image provided below and upload it on stage `GENAI_IMAGES`. [Learn how](https://docs.snowflake.com/en/user-guide/data-load-local-file-system-stage-ui?_fsi=oZm563yp&_fsi=oZm563yp#upload-files-onto-a-named-internal-stage)

Sample image:
- https://sfquickstarts.s3.us-west-1.amazonaws.com/misc/images/other/sample-img-3.jpg

*Note: Sample image provided courtesy of [Dash](https://natureunraveled.com/).*

Step 9: Preview image analysis produced on the new sample image

In [ ]:
select * from image_analysis;

Step 10: Suspend task

In [ ]:
ALTER TASK image_analysis_task SUSPEND;